# “dx extract_assay expression” in Python
<hr/>
***As-Is Software Disclaimer***

This content in this repository is delivered “As-Is”. Notwithstanding anything to the contrary, DNAnexus will have no warranty, support, liability or other obligations with respect to Materials provided hereunder.

<hr/>
This notebook demonstrates usage of the dx command `extract_assay expression` to:

* Filter expression data in a DNAnexus dataset
* Generate an expression matrix from a DNAnexus dataset

## Preparing your environment
### Launch spec:

* App name: JupyterLab with Python, R, Stata, ML
* Kernel: Python
* Instance type: mem1_ssd1_v2_x2
* Cost: < $0.1
* Runtime: < 2 min
* Data description: Input for this notebook is a v3.0 Dataset or Cohort object ID.

In [ ]:
# For dx extract_assay expression, dxpy must be v0.364.0 or greater
# However, a more recent version of dxpy on PyPI may already be available
# and installed, making the below "pip" install unecessary.
!pip3 install -U "dxpy>=0.364.0"

In [ ]:
import subprocess
import dxpy
import json

### Print help message

In [ ]:
cmd = ["dx", "extract_assay", "expression", "--help"]
help_text = subprocess.check_output(cmd)
print(help_text.decode(), end="\n")

### Assign environment variables

In [ ]:
# The referenced dataset is not a public dataset and listed here only to demonstrate as an example input.
# The user will need to supply a permissible and valid project ID and record ID

# Assign a project qualified dataset record, project-id:record-id
dataset = "project-G5Bzk5806j8V7PXB678707bv:record-GYPg9Jj06j8pp3z41682J23p"

### Explore the expression assays of a dataset

Check which expression assays are available in the dataset using the `--list-assays` flag.

In [ ]:
cmd = ["dx", "extract_assay", "expression", dataset, "--list-assays"]
output = subprocess.check_output(cmd)
print(output.decode(), end="\n")

### Retrieve expression data from the dataset

Data may be retrieved using the option, `--retrieve-expression`. The option accepts a JSON object as input for filtering purposes, where the object is supplied either as a \*.json file or as a string. For additional help, use `--json-help` trailing `--retrieve-expression` for both a template and a list of filters available to the respective option.

In [ ]:
cmd = ["dx", "extract_assay", "expression", dataset, "--retrieve-expression", "--json-help"]
output = subprocess.check_output(cmd)
print(output.decode(), end="\n")

#### Example on how to create a JSON of filters to retrieve data

JSON may be created in many ways, including writing the object to a file.

In [ ]:
filter_dict = {
    "location": [
        {
            "chromosome": "1",
            "starting_position": "1",
            "ending_position": "250000000"
        }
    ],
}

filter_json = json.dumps(filter_dict)

with open("filter.json", "w") as op:
    op.write(filter_json)

In [ ]:
cmd = ["cat", "filter.json"]
output = subprocess.check_output(cmd)
print(output.decode(), end="\n")

#### Use the generated JSON file and retrieve expression data

List feature names and related information based on the variant filters mentioned in the filter.json. The output data can be printed to STDOUT or written to a file. Here we print it to STDOUT.

In [ ]:
cmd = ["dx", "extract_assay", "expression", dataset, "--retrieve-expression",
       "--filter-json-file", "filter.json", "-o", "-"]
output = subprocess.check_output(cmd)
print(output.decode(), end="\n")

#### Add additional fields to output

For a full list of fields available for output, use the flag, `--additional-fields-help`.

In [ ]:
cmd = ["dx", "extract_assay", "expression", dataset, "--additional-fields-help"]
output = subprocess.check_output(cmd)
print(output.decode(), end="\n")

Specify additional fields using `--additional-fields`

In [ ]:
cmd = ["dx", "extract_assay", "expression", dataset,
       "--retrieve-expression","--filter-json-file", "filter.json",
       "--additional-fields", "start,end,strand",
       "-o", "-"]
output = subprocess.check_output(cmd)
print(output.decode(), end="\n")

## Generate an expression matrix from a dataset
The extract_assay expression function can be configured to output an expression matrix rather than its standard output.  This matrix contains a column for sample_id, and an additional column for every feature_id in the dataset.  There is only one sample_id per row.  Note, filtering is limited when creating an expression matrix and additional fields are not included in the expression matrix.

In [ ]:
cmd = ["dx", "extract_assay", "expression", dataset,
       "--retrieve-expression",
       "--filter-json-file", "filter.json",
       "--expression-matrix",
       "-o", "-"]
output = subprocess.check_output(cmd)
print(output.decode(), end="\n")

## Generate SQL for data extraction

Alternatively, we can generate a SQL query instead of extracting the data. This SQL query can be run in a spark-enabled cluster to retrieve data. To generate a SQL query, we use the `--sql` flag and print it on STDOUT.

In [ ]:
cmd = ["dx", "extract_assay", "expression", dataset,
       "--retrieve-expression",
       "--filter-json-file", "filter.json",
       "--sql","-o", "-"]
output = subprocess.check_output(cmd)
print(output.decode(), end="\n")

## Return data from a specific assay

We provide multi-assay support, where there may be one, or many of the same assay type. If multiple assays of the same type exist in a dataset, it is possible to specify the exact assay to query from using the argument, --assay-name. Default functionality is to simply return data from the first eligible assay in the dataset.

In [ ]:
assay_name = "enst_short_multiple_assays_sample_2_enst"
cmd = ["dx", "extract_assay", "expression", dataset,
       "--assay-name", assay_name,
       "--retrieve-expression",
       "--filter-json-file", "filter.json",
        "-o", "-"]
output = subprocess.check_output(cmd)
print(output.decode(), end="\n")